In [1]:
import pandas as pd
import numpy as np
import torch
import pickle
import os
import json
import gc
from torch.distributions import Bernoulli
from torch.optim import LBFGS
from tqdm import tqdm
from scipy.stats import pearsonr
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor
from multiprocessing import Manager
import multiprocessing as mp

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tueplots import bundles
bundles.icml2024()

from torchmetrics import AUROC
auroc = AUROC(task="binary")

import warnings
warnings.filterwarnings("ignore")

torch.manual_seed(0)

device = "cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

def visualize_response_matrix(results, value, filename):
    # Extract the groups labels in the order of the columns
    group_values = results.columns.get_level_values("scenario")

    # Identify the boundaries where the group changes
    boundaries = []
    for i in range(1, len(group_values)):
        if group_values[i] != group_values[i - 1]:
            boundaries.append(i - 0.5)  # using 0.5 to place the line between columns

    # Visualize the results with a matrix: red is 0, white is -1 and blue is 1
    cmap = mcolors.ListedColormap(["white", "red", "blue"])
    bounds = [-1.5, -0.5, 0.5, 1.5]
    norm = mcolors.BoundaryNorm(bounds, cmap.N)

    # Calculate midpoints for each group label
    groups_list = list(group_values)
    group_names = []
    group_midpoints = []
    current_group = groups_list[0]
    start_index = 0
    for i, grp in enumerate(groups_list):
        if grp != current_group:
            midpoint = (start_index + i - 1) / 2.0
            group_names.append(current_group)
            group_midpoints.append(midpoint)
            current_group = grp
            start_index = i
    # Add the last group
    midpoint = (start_index + len(groups_list) - 1) / 2.0
    group_names.append(current_group)
    group_midpoints.append(midpoint)

    # Define the minimum spacing between labels (e.g., 100 units)
    min_spacing = 100
    last_label_pos = -float("inf")
    # Plot the matrix
    with plt.rc_context(bundles.icml2024(usetex=True, family="serif")):
        fig, ax = plt.subplots(figsize=(20, 10))
        cax = ax.matshow(value, aspect="auto", cmap=cmap, norm=norm)

        # Add vertical lines at each boundary
        for b in boundaries:
            ax.axvline(x=b, color="black", linewidth=0.25, linestyle="--", alpha=0.5)
        
        # Add group labels above the matrix, only if they're spaced enough apart
        for name, pos in zip(group_names, group_midpoints):
            if pos - last_label_pos >= min_spacing:
                ax.text(pos, -5, name, ha='center', va='bottom', rotation=90, fontsize=3)
                last_label_pos = pos

        # Add model labels on the y-axis
        ax.set_yticks(range(len(results.index)))
        ax.set_yticklabels(results.index, fontsize=3)

        # Add a colorbar
        cbar = plt.colorbar(cax)
        cbar.set_ticks([-1, 0, 1])
        cbar.set_ticklabels(["-1", "0", "1"])
        plt.savefig(filename, dpi=600, bbox_inches="tight")
        plt.close()

def trainer(parameters, optim, closure, n_iter=100, verbose=True):
    pbar = tqdm(range(n_iter)) if verbose else range(n_iter)
    for iteration in pbar:
        if iteration > 0:
            previous_parameters = [p.clone() for p in parameters]
            previous_loss = loss.clone()
        
        loss = optim.step(closure)
        
        if iteration > 0:
            d_loss = (previous_loss - loss).item()
            d_parameters = sum(
                torch.norm(prev - curr, p=2).item()
                for prev, curr in zip(previous_parameters, parameters)
            )
            grad_norm = sum(torch.norm(p.grad, p=2).item() for p in parameters if p.grad is not None)
            if verbose:
                pbar.set_postfix({"grad_norm": grad_norm, "d_parameter": d_parameters, "d_loss": d_loss})
            
            if d_loss < 1e-5 and d_parameters < 1e-5 and grad_norm < 1e-5:
                break
    return parameters

def compute_auc(probs, data, train_idtor, test_idtor):
    train_probs = probs[train_idtor.bool()]
    test_probs = probs[test_idtor.bool()]
    train_labels = data[train_idtor.bool()]
    test_labels = data[test_idtor.bool()]
    train_auc = auroc(train_probs, train_labels)
    test_auc = auroc(test_probs, test_labels)
    print(f"train auc: {train_auc}")
    print(f"test auc: {test_auc}")
    
    return train_auc, test_auc

def compute_cttcorr(probs, data, train_idtor, test_idtor):
    train_probs  = probs.clone()
    test_probs   = probs.clone()
    train_labels = data.clone()
    test_labels  = data.clone()

    train_mask = ~train_idtor.bool()
    train_probs[train_mask]  = float('nan')
    train_labels[train_mask] = float('nan')

    test_mask = ~test_idtor.bool()
    test_probs[test_mask]   = float('nan')
    test_labels[test_mask]  = float('nan')
    
    train_prob_ctt = torch.nanmean(train_probs, dim=1).detach().cpu().numpy()
    train_label_ctt = torch.nanmean(train_labels, dim=1).detach().cpu().numpy()
    train_mask = ~np.isnan(train_prob_ctt) & ~np.isnan(train_label_ctt)
    train_cttcorr = pearsonr(train_prob_ctt[train_mask], train_label_ctt[train_mask]).statistic
    
    test_prob_ctt = torch.nanmean(test_probs, dim=1).detach().cpu().numpy()
    test_label_ctt = torch.nanmean(test_labels, dim=1).detach().cpu().numpy()
    test_mask = ~np.isnan(test_prob_ctt) & ~np.isnan(test_label_ctt)
    test_cttcorr = pearsonr(test_prob_ctt[test_mask], test_label_ctt[test_mask]).statistic
    
    print(f"train cttcorr: {train_cttcorr}")
    print(f"test cttcorr: {test_cttcorr}")

    return train_cttcorr, test_cttcorr

/anaconda/envs/reeval/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
with open(f"../data/resmat.pkl", "rb") as f:
    results = pickle.load(f)

dtype = torch.float64 if device.startswith("cuda") else torch.float32

# data_withnan, missing=nan
# data_withneg1, missing=-1
# data_with0, missing=0
data_withnan = torch.tensor(results.values, dtype=dtype, device=device)
data_idtor = (~torch.isnan(data_withnan)).to(dtype)
data_withneg1 = data_withnan.nan_to_num(nan=-1.0)
data_with0 = data_withneg1 * data_idtor
data_with0 = data_with0.nan_to_num(nan=0.0)
n_test_takers, n_items = data_with0.shape
scenarios = results.columns.get_level_values("scenario").unique()

In [3]:
data_with0.unique()

tensor([0., 1.], device='cuda:0', dtype=torch.float64)

Grid search for suitable dimensions

In [ ]:
import os

results_dim = []

for n_dimensions in range(2, 12):
    print(f"\n=== Running MIRT with n_dimensions={n_dimensions} ===")
    # --- Setup ---
    n_test_takers = 183
    n_items = 78712

    # Mask generation
    valid_condition = False
    trial = 0
    while not valid_condition:
        train_idtor = torch.bernoulli(data_idtor * 0.8).int()
        test_idtor = data_idtor - train_idtor
        valid_condition = (train_idtor.sum(axis=1) != 0).all() and (train_idtor.sum(axis=0) != 0).all()
        trial += 1

    # Stage 1: Item calibration
    n_mc_samples = 150
    thetas_nuisance = torch.randn(n_mc_samples, n_test_takers, n_dimensions, device=device)
    a_params = torch.randn(n_items, n_dimensions, requires_grad=True, device=device)
    ds = torch.randn(n_items, requires_grad=True, device=device)
    B = 50000
    for i in tqdm(range(0, n_items, B), desc=f"Stage 1: Calibrating Item Batches (dim={n_dimensions})"):
        current_B = min(B, n_items - i)
        a_params_batch = a_params[i:i+current_B].clone().detach().requires_grad_(True)
        ds_batch = ds[i:i+current_B].clone().detach().requires_grad_(True)
        data_batch = data_with0[:, i:i+current_B]
        train_idtor_batch = train_idtor[:, i:i+current_B]
        optim_items = LBFGS([a_params_batch, ds_batch], lr=0.1, max_iter=20, history_size=10, line_search_fn="strong_wolfe")
        def closure_items():
            optim_items.zero_grad()
            a_params_constrained = torch.clamp(a_params_batch, min=0)
            logits = torch.matmul(thetas_nuisance, a_params_constrained.T) - ds_batch[None, None, :]
            probs = torch.sigmoid(logits)
            log_likelihoods = Bernoulli(probs=probs).log_prob(data_batch[None, :, :]) * train_idtor_batch[None, :, :]
            loss = -log_likelihoods.sum() / (train_idtor_batch.sum() * n_mc_samples)
            loss.backward()
            return loss
        optim_items.step(closure_items)
        a_params.data[i:i+current_B] = a_params_batch.data
        ds.data[i:i+current_B] = ds_batch.data

    a_params_calibrated_unrotated = a_params.detach()
    ds_calibrated = ds.detach()
    a_params_np = a_params_calibrated_unrotated.cpu().numpy()
    rotator = Rotator(method='varimax')
    rotated_a_params = rotator.fit_transform(a_params_np)
    a_params_calibrated = torch.tensor(rotated_a_params, device=device, dtype=torch.float32)

    # Stage 2: Person calibration
    thetas = torch.randn(n_test_takers, n_dimensions, requires_grad=True, device=device)
    optim_thetas = LBFGS([thetas], lr=0.1, max_iter=20, history_size=10, line_search_fn="strong_wolfe")
    def closure_thetas():
        optim_thetas.zero_grad()
        thetas_constrained = torch.clamp(thetas, min=0)
        a_params_constrained = torch.clamp(a_params_calibrated, min=0)
        logits = torch.matmul(thetas_constrained, a_params_constrained.T) - ds_calibrated[None, :]
        probs = torch.sigmoid(logits)
        log_likelihoods = Bernoulli(probs=probs).log_prob(data_with0) * train_idtor
        loss = -log_likelihoods.sum() / train_idtor.sum()
        loss.backward()
        return loss
    optim_thetas.step(closure_thetas)
    thetas_final = thetas.detach()
    a_params_final = a_params_calibrated
    ds_final = ds_calibrated

    # Evaluation
    thetas_constrained_final = torch.clamp(thetas_final, min=0)
    a_params_constrained_final = torch.clamp(a_params_final, min=0)
    logits = torch.matmul(thetas_constrained_final, a_params_constrained_final.T) - ds_final[None, :]
    probs = torch.sigmoid(logits)
    train_auc, test_auc = compute_auc(probs, data_with0, train_idtor, test_idtor)
    train_cttcorr, test_cttcorr = compute_cttcorr(probs, data_with0, train_idtor, test_idtor)

    # Model fit indices
    num_item_params = n_items * (n_dimensions + 1)
    num_person_params = n_test_takers * n_dimensions
    k = num_item_params + num_person_params
    n_obs = train_idtor.sum().item()
    with torch.no_grad():
        log_likelihood = Bernoulli(probs=probs).log_prob(data_with0) * train_idtor
        total_log_likelihood = log_likelihood.sum()
    aic = 2 * k - 2 * total_log_likelihood
    bic = k * torch.log(torch.tensor(n_obs, dtype=torch.float32)) - 2 * total_log_likelihood

    # Save results
    results_dim.append({
        "n_dimensions": n_dimensions,
        "train_auc": float(train_auc),
        "test_auc": float(test_auc),
        "train_cttcorr": float(train_cttcorr),
        "test_cttcorr": float(test_cttcorr),
        "log_likelihood": float(total_log_likelihood),
        "AIC": float(aic),
        "BIC": float(bic)
    })
    print(f"n_dim={n_dimensions}: train_auc={train_auc:.4f}, test_auc={test_auc:.4f}, train_cttcorr={train_cttcorr:.4f}, test_cttcorr={test_cttcorr:.4f}, AIC={aic:.2f}, BIC={bic:.2f}")

# Save all results to file
os.makedirs("../result", exist_ok=True)
import json
with open("../result/mirt_dim_results.json", "w") as f:
    json.dump(results_dim, f, indent=2)


=== Running MIRT with n_dimensions=2 ===


Stage 1: Calibrating Item Batches (dim=2): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:14<00:00,  7.48s/it]


train auc: 0.8111507892608643
test auc: 0.7964804172515869
train cttcorr: 0.7075257534721937
test cttcorr: 0.6911825414658618
n_dim=2: train_auc=0.8112, test_auc=0.7965, train_cttcorr=0.7075, test_cttcorr=0.6912, AIC=4902541.36, BIC=8039973.86

=== Running MIRT with n_dimensions=3 ===


Stage 1: Calibrating Item Batches (dim=3): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:14<00:00,  7.50s/it]


train auc: 0.8115150928497314
test auc: 0.7962170243263245
train cttcorr: 0.7488648382438651
test cttcorr: 0.7330264514074442
n_dim=3: train_auc=0.8115, test_auc=0.7962, train_cttcorr=0.7489, test_cttcorr=0.7330, AIC=5057767.69, BIC=9242054.19

=== Running MIRT with n_dimensions=4 ===


...

train auc: 0.8176456689834595
test auc: 0.8021864891052246
train cttcorr: 0.8291225554835577
test cttcorr: 0.815854001696239
n_dim=9: train_auc=0.8176, test_auc=0.8022, train_cttcorr=0.8291, test_cttcorr=0.8159, AIC=5995623.37, BIC=16459944.37

=== Running MIRT with n_dimensions=10 ===


Stage 1: Calibrating Item Batches (dim=10): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:15<00:00,  7.54s/it]


train auc: 0.8153287172317505
test auc: 0.7995522022247314
train cttcorr: 0.8133266209595373
test cttcorr: 0.793146471314127
n_dim=10: train_auc=0.8153, test_auc=0.7996, train_cttcorr=0.8133, test_cttcorr=0.7931, AIC=6135275.69, BIC=17646331.69

=== Running MIRT with n_dimensions=11 ===


Stage 1: Calibrating Item Batches (dim=11): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:15<00:00,  7.54s/it]


train auc: 0.8185120224952698
test auc: 0.8026008009910583
train cttcorr: 0.7942789220727466
test cttcorr: 0.7823387281679738
n_dim=11: train_auc=0.8185, test_auc=0.8026, train_cttcorr=0.7943, test_cttcorr=0.7823, AIC=6281398.70, BIC=18838912.70


In [9]:
# <<< Make sure you have factor_analyzer installed: pip install factor_analyzer >>>
import torch
import numpy as np
import pandas as pd
from torch.distributions import Bernoulli
from torch.optim import LBFGS
from tqdm import tqdm
from factor_analyzer.rotator import Rotator
import gc

# Assume prerequisite data tensors are loaded...
# device, data_with0, data_idtor, n_test_takers, n_items

# ====================================================================================
# 1. SETUP & DATA SIMULATION
# ====================================================================================

n_test_takers = 183
n_items = 78712
# <<< We set the number of dimensions we want to DISCOVER >>>
n_dimensions = 9

# ====================================================================================
# 3. MIRT MODEL TRAINING (Exploratory Approach)
# ====================================================================================

# Apply random train/test mask to the matrix
valid_condition = False
trial = 0
while not valid_condition:
    train_idtor = torch.bernoulli(data_idtor * 0.8).int()
    test_idtor = data_idtor - train_idtor
    valid_condition = (train_idtor.sum(axis=1) != 0).all() and (train_idtor.sum(axis=0) != 0).all()
    print(f"trial {trial} valid condition: {valid_condition}")
    trial += 1

# --- STAGE 1: Fit Item Parameters (a_params and ds) ---
print("Starting Stage 1: Fitting item parameters (a_params and ds)...")

n_mc_samples = 150
thetas_nuisance = torch.randn(n_mc_samples, n_test_takers, n_dimensions, device=device)

a_params = torch.randn(n_items, n_dimensions, requires_grad=True, device=device)
ds = torch.randn(n_items, requires_grad=True, device=device)

B = 50000
for i in tqdm(range(0, n_items, B), desc="Stage 1: Calibrating Item Batches"):
    current_B = min(B, n_items - i)
    a_params_batch = a_params[i:i+current_B].clone().detach().requires_grad_(True)
    ds_batch = ds[i:i+current_B].clone().detach().requires_grad_(True)

    data_batch = data_with0[:, i:i+current_B]
    train_idtor_batch = train_idtor[:, i:i+current_B]
    
    optim_items = LBFGS([a_params_batch, ds_batch], lr=0.1, max_iter=20, history_size=10, line_search_fn="strong_wolfe")

    def closure_items():
        optim_items.zero_grad()
        
        # <<< CHANGE: Removed the Q-matrix masking. We now estimate a dense matrix. >>>
        a_params_constrained = torch.clamp(a_params_batch, min=0)
        
        logits = torch.matmul(thetas_nuisance, a_params_constrained.T) - ds_batch[None, None, :]
        probs = torch.sigmoid(logits)
        
        log_likelihoods = Bernoulli(probs=probs).log_prob(data_batch[None, :, :]) * train_idtor_batch[None, :, :]
        loss = -log_likelihoods.sum() / (train_idtor_batch.sum() * n_mc_samples)
        
        loss.backward()
        return loss

    optim_items.step(closure_items)
    
    a_params.data[i:i+current_B] = a_params_batch.data
    ds.data[i:i+current_B] = ds_batch.data

a_params_calibrated_unrotated = a_params.detach()
ds_calibrated = ds.detach()
print("Stage 1 Finished (unrotated).")

# <<< NEW STEP: Perform Post-Hoc Factor Rotation to create the latent Q-matrix >>>
print("\n--- Performing Factor Rotation ---")
a_params_np = a_params_calibrated_unrotated.cpu().numpy()
rotator = Rotator(method='varimax')
rotated_a_params = rotator.fit_transform(a_params_np)
print("Rotation complete. The rotated matrix is now the interpretable latent Q-matrix.")
a_params_calibrated = torch.tensor(rotated_a_params, device=device, dtype=torch.float32)


# --- STAGE 2: Fit Person Parameters (thetas) ---
print("\nStarting Stage 2: Fitting person parameters (thetas)...")

thetas = torch.randn(n_test_takers, n_dimensions, requires_grad=True, device=device)
optim_thetas = LBFGS([thetas], lr=0.1, max_iter=20, history_size=10, line_search_fn="strong_wolfe")

def closure_thetas():
    optim_thetas.zero_grad()
    
    thetas_constrained = torch.clamp(thetas, min=0)
    # <<< CHANGE: Use the new rotated a_params without Q-matrix masking >>>
    a_params_constrained = torch.clamp(a_params_calibrated, min=0)
    
    logits = torch.matmul(thetas_constrained, a_params_constrained.T) - ds_calibrated[None, :]
    probs = torch.sigmoid(logits)
    
    log_likelihoods = Bernoulli(probs=probs).log_prob(data_with0) * train_idtor
    loss = -log_likelihoods.sum() / train_idtor.sum()
    
    loss.backward()
    return loss

optim_thetas.step(closure_thetas)

thetas_final = thetas.detach()
a_params_final = a_params_calibrated # Use the rotated parameters
ds_final = ds_calibrated
print("Stage 2 Finished. Training complete.")

# --- 4. EVALUATION ---
thetas_constrained_final = torch.clamp(thetas_final, min=0)
# <<< CHANGE: Use final rotated a_params without Q-matrix masking >>>
a_params_constrained_final = torch.clamp(a_params_final, min=0)

logits = torch.matmul(thetas_constrained_final, a_params_constrained_final.T) - ds_final[None, :]
probs = torch.sigmoid(logits)

train_auc, test_auc = compute_auc(probs, data_with0, train_idtor, test_idtor)
train_cttcorr, test_cttcorr = compute_cttcorr(probs, data_with0, train_idtor, test_idtor)

# --- 5. POST-HOC ANALYSIS ---
print("\n--- Post-Hoc Analysis ---")

# Ensure the final thetas are on the CPU and converted to a NumPy array
thetas_np = thetas_final.cpu().numpy()

# Calculate the correlation matrix
# rowvar=False is important: it tells numpy that your variables are columns, not rows.
theta_correlation_matrix = np.corrcoef(thetas_np, rowvar=False)

print("Theta Correlation Matrix:")
print(theta_correlation_matrix)
print("\nInterpretation:")
print("- Values close to 0 mean the dimensions are distinct.")
print("- Values close to 1 or -1 mean the dimensions are highly related.")
import torch

# --- 6. MODEL FIT CALCULATION ---
print("\n--- Model Fit Indices ---")

# 1. Define Model Complexity (k = number of parameters)
num_item_params = n_items * (n_dimensions + 1)  # (a * K) + d
num_person_params = n_test_takers * n_dimensions # theta * K
k = num_item_params + num_person_params
print(f"Number of parameters (k): {k}")

# 2. Define Number of Observations (n = number of responses in training set)
n = train_idtor.sum().item()
print(f"Number of observations (n): {n}")

# 3. Calculate Final Log-Likelihood on the Training Data
with torch.no_grad(): # We don't need to compute gradients here
    # Use the final, constrained parameters
    thetas_constrained_final = torch.clamp(thetas_final, min=0)
    
    # <<< CORRECTED LINE: Removed the Q-matrix masking >>>
    a_params_constrained_final = torch.clamp(a_params_final, min=0)
    
    # Calculate probabilities for the training data
    logits = torch.matmul(thetas_constrained_final, a_params_constrained_final.T) - ds_final[None, :]
    probs = torch.sigmoid(logits)
    
    # Calculate the log-likelihood only on the training responses
    log_likelihood = Bernoulli(probs=probs).log_prob(data_with0) * train_idtor
    total_log_likelihood = log_likelihood.sum()

# 4. Calculate AIC and BIC
aic = 2 * k - 2 * total_log_likelihood
bic = k * torch.log(torch.tensor(n, dtype=torch.float32)) - 2 * total_log_likelihood

print(f"\nTotal Log-Likelihood: {total_log_likelihood.item():.2f}")
print(f"AIC: {aic.item():.2f}")
print(f"BIC: {bic.item():.2f}")
print("\nReminder: Lower AIC/BIC values indicate a better model fit.")

trial 0 valid condition: True
Starting Stage 1: Fitting item parameters (a_params and ds)...


Stage 1: Calibrating Item Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:15<00:00,  7.51s/it]


Stage 1 Finished (unrotated).

--- Performing Factor Rotation ---
Rotation complete. The rotated matrix is now the interpretable latent Q-matrix.

Starting Stage 2: Fitting person parameters (thetas)...
Stage 2 Finished. Training complete.
train auc: 0.8181132674217224
test auc: 0.8022111654281616
train cttcorr: 0.8102029631594286
test cttcorr: 0.786183379343087

--- Post-Hoc Analysis ---
Theta Correlation Matrix:
[[ 1.          0.13031827  0.18375701  0.10735548  0.1306726   0.17823001
   0.08648952  0.22196541  0.29091689]
 [ 0.13031827  1.          0.22138605  0.14695814  0.06316584  0.06822706
   0.14489865  0.23206715  0.11745934]
 [ 0.18375701  0.22138605  1.          0.1213672   0.13798604  0.20337862
   0.12705691  0.13689088  0.17047772]
 [ 0.10735548  0.14695814  0.1213672   1.          0.09469717  0.11417649
   0.20637249 -0.01874886  0.01321276]
 [ 0.1306726   0.06316584  0.13798604  0.09469717  1.          0.23768495
   0.11290381  0.10915652  0.11194   ]
 [ 0.17823001  0.

In [13]:
import pandas as pd
import numpy as np

# --- PREREQUISITES ---
# Assume you have these variables from your previous script:
# 1. rotated_a_params: The (78712, 9) numpy array of rotated loadings.
# 2. An item_master_df that maps item_id to scenario, like we created before.
#    If you don't have it, you can re-create it:
test_data_df = pd.read_pickle("../data/resmat.pkl")
item_master_df = pd.DataFrame({
	'item_id': range(78712),
	'scenario': test_data_df.columns.get_level_values('scenario')
})
# ----------------------------------------------------

print("--- Interpreting 9 Discovered Dimensions ---")

# Create a DataFrame of the loadings
dim_names = [f'Dim_{i+1}' for i in range(n_dimensions)]
loadings_df = pd.DataFrame(rotated_a_params, columns=dim_names)

# Combine with scenario information
interpretation_df = pd.concat([item_master_df, loadings_df], axis=1)

# Set the loading threshold for interpretation
loading_threshold = 0.40

for dim_name in dim_names:
    print(f"\n--- Analysis for {dim_name} ---")
    
    # Find items that load strongly on this dimension
    strong_loaders = interpretation_df[abs(interpretation_df[dim_name]) >= loading_threshold]
    
    if len(strong_loaders) == 0:
        print("No datasets strongly load on this dimension.")
        continue
    
    # Count which scenarios are most represented in the strong loaders
    scenario_counts = strong_loaders['scenario'].value_counts()
    
    print(f"Top contributing scenarios for {dim_name}:")
    print(scenario_counts.head(5))

--- Interpreting 9 Discovered Dimensions ---

--- Analysis for Dim_1 ---
Top contributing scenarios for Dim_1:
scenario
civil_comments    10293
mmlu               4664
wikifact           1979
air_bench_2024     1732
imdb               1270
Name: count, dtype: int64

--- Analysis for Dim_2 ---
Top contributing scenarios for Dim_2:
scenario
civil_comments    9997
mmlu              4691
wikifact          1924
air_bench_2024    1724
imdb              1320
Name: count, dtype: int64

--- Analysis for Dim_3 ---
Top contributing scenarios for Dim_3:
scenario
civil_comments    10065
mmlu               4574
wikifact           1905
air_bench_2024     1747
babi_qa            1255
Name: count, dtype: int64

--- Analysis for Dim_4 ---
Top contributing scenarios for Dim_4:
scenario
civil_comments    10080
mmlu               4661
wikifact           1936
air_bench_2024     1751
imdb               1278
Name: count, dtype: int64

--- Analysis for Dim_5 ---
Top contributing scenarios for Dim_5:
scenario
c